#### Setup env and jupyter kernel

In [1]:
!uv run $DATA_ROOT/ml-lab/setup_kernel.py diffusers-lab

Using CPython 3.13.11
Creating virtual environment at: .venv
Resolved 163 packages in 4.04s                                       
Prepared 12 packages in 2.19s                                            
Installed 134 packages in 266ms1.0a20260118                 
 + accelerate==1.13.0.dev0 (from git+https://github.com/huggingface/accelerate@38dadd9537d3cb43e3149964c06a3f011b40aa6d)
 + anyio==4.12.1
 + argon2-cffi==25.1.0
 + argon2-cffi-bindings==25.1.0
 + arrow==1.4.0
 + asttokens==3.0.1
 + async-lru==2.1.0
 + attrs==25.4.0
 + babel==2.17.0
 + beautifulsoup4==4.14.3
 + bleach==6.3.0
 + certifi==2022.12.7
 + cffi==2.0.0
 + charset-normalizer==2.1.1
 + click==8.3.1
 + comm==0.2.3
 + compel==2.3.1
 + contourpy==1.3.3
 + cycler==0.12.1
 + debugpy==1.8.19
 + decorator==5.2.1
 + defusedxml==0.7.1
 + diffusers==0.37.0.dev0 (from git+https://github.com/huggingface/diffusers@ec376293714f269947f6d9d8a572bd73040bc1a0)
 + diffusers-lab==0.1.0 (from file:///work/ml-lab/diffusers-lab)
 + executing

#### SDXL

Initial setup

In [1]:
import os
from IPython.display import display
import torch
from diffusers import StableDiffusionXLPipeline

# Disable MIOpen on gfx1151
if os.environ.get("RUNTIME_TYPE", "") == "rocm-gfx1151":
    torch.backends.cudnn.enabled = False

work_dir = os.environ.get('DATA_ROOT', '/work')
%cd {work_dir}
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

/work


Load models

In [2]:
model = "checkpoints/JANKUTrainedNoobaiRouwei_v60.safetensors"
model_path = f"{work_dir}/models/{model}"

pipeline = StableDiffusionXLPipeline.from_single_file(
    model_path,
    use_safetensors=True,
    disable_mmap=True,
    )
pipeline = pipeline.to("cuda", dtype=torch.float16)
pipeline


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.37.0.dev0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
from diffusers import EulerAncestralDiscreteScheduler
from diffusers_lab.prompting import get_il_prompts 
from diffusers_lab.conditioning import get_sdxl_prompt_embeds
from typing import Any

pos, _, neg, _ = get_il_prompts("""
1girl, solo,
full body, standing,
straight on,
clothed, safe,
looking at viewer, smiling,
hatsune miku,
white shirt, short sleeves
white skirt, pleated skirt, white thighhighs
outdoors, park, cloud, sky, noon
""", reality=False)
_ = display((pos, neg))

prompt_embeds = get_sdxl_prompt_embeds(pipeline, pos, neg)

pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipeline.scheduler.config,
    use_beta_sigmas = True,
    timestep_spacing = "linspace"
    )
generator = torch.Generator(device="cuda").manual_seed(78678)
params: dict[str, Any] = {
    "num_inference_steps": 20,
    "guidance_scale": 2.5,
    "width": 1024,
    "height": 1536
}
params.update(prompt_embeds)
#params["generator"] = generator

image = pipeline(**params)[0][0] # pyright: ignore[reportOptionalSubscript, reportIndexIssue]
image

('masterpiece, best quality, amazing quality, 4k, very aesthetic, high resolution, ultra-detailed, absurdres, newest, scenery, 1girl, solo, full body, standing, straight on, clothed, safe, looking at viewer, smiling, hatsune miku, white shirt, short sleeves, white skirt, pleated skirt, white thighhighs, outdoors, park, cloud, sky, noon, BREAK, depth of field',
 '1girl, solo, full body, standing, straight on, clothed, safe, looking at viewer, smiling, hatsune miku, white shirt, short sleeves, white skirt, pleated skirt, white thighhighs, outdoors, park, cloud, sky, noon, monochrome, modern, recent, old, oldest, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured, long body, lowres, bad anatomy, (bad hands, missing fingers, extra fingers, extra digits, fewer digits:1.2), cropped, very displeasing, (worst quality, bad quality:1.2), sketch, jpeg artifacts, signature, watermark, username, (censored, bar_censor, mosaic_censor:1.2), conjoi

Token indices sequence length is longer than the specified maximum sequence length for this model (90 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (90 > 77). Running this sequence through the model will result in indexing errors


  0%|          | 0/20 [00:00<?, ?it/s]

/work/ml-lab/diffusers-lab/.venv/lib/python3.13/site-packages/diffusers/pipelines/stable_diffusion_xl/pipeline_stable_diffusion_xl.py:748: FutureWarning: `upcast_vae` is deprecated and will be removed in version 1.0.0. `upcast_vae` is deprecated. Please use `pipe.vae.to(torch.float32)`. For more details, please refer to: https://github.com/huggingface/diffusers/pull/12619#issue-3606633695.
  deprecate(
